Welcome to my project !
in this following notebook you would encounter some practices about:
reciving JSON response from REST API of a mobile game - Disney Sorcerers Arena.
translating the response to normalized tables with pandas.


That an RPG Game which focus on wide roster of characters and offer PVP and PVE modes.
The purpose of this project is to create rdb for analysis and bots creation.

I wish to empower my skills in engineering.
in this notebook you will find practice about how to extract crucial data about the character power.

In [ ]:
import numpy as np,pandas as pd,json,sqlite3,requests
from jsonpath_ng import jsonpath
from jsonpath_ng.ext import parse
from pandas.io.json import json_normalize
pd.options.display.max_columns = 99

with open('/Users/mac/Documents/Coding/Projects/DSA/Resources/characters_in_dsa.json', 'r') as char_data:
    chars_json = json.load(char_data)



After loading the source file as python object, I found this meta deta:
- there are 176 main arrays. each array holds all the data over a character.
let's remind that each array evaluate to more arrays of key-value, and value can become another array (hello nested).

I'd use serieses and DataFrames constantly to avoid memory and time consuming of slower classes in python.

In [ ]:
print(type(chars_json)) #dict
print(len(chars_json)) #176
toons_ids = pd.Series(chars_json.keys())
print(type(toons_ids))
print(toons_ids[0])

First, We need to figure out what we can extract, and looking at the first level We only got the toons ID's. let's dive deeper. I want to keep my script as reusable as possible so I try to avoid the singular data usage. I took all the id's and created a list of the values, so I can take the first position and work with that.

That some more helpful keys. I want to construct a table of the toons. Thanks to my experience in the game my data will be:
ID | NameKey | StatProgressionTable | Obtainable | UnlockRarity | *Categories

The categories level is another dict that needs to be flattened. I wish to make it:
ID | NameKey | StatProgressionTable | Obtainable | UnlockRarity | Class | Fiction | Story.

Let's see what's going on there.

Thanks to the versatility of JSON in Python we can traverse through our object to seperate the categories and manage them one by one.

In [ ]:
with open('/Users/mac/Documents/Coding/Projects/DSA/Resources/game_data_in_dsa.json', 'r') as game_data_file:
    game_data = json.load(game_data_file)
unit_categories = game_data['unit_categories']

Let's sum so far;
We managed to create a DataFrame that holds the crucial data and the categories as a list.
As We know, relational table cannot hold more then one value, thereby We need to fix columns.
We will go with Class, Fiction, Story and Type, there are more tags but as core-table We nned general data.

Next, We are gonna fetch some data from our tables to be able to compare to the given categories list.

In [ ]:
types = []
types_id = []
with sqlite3.connect('/Users/mac/Documents/Coding/Projects/DSA/Resources/dsa.db') as conn:
    curr = conn.cursor()
    types_id = curr.execute('SELECT category_id  FROM toons_categories where category_type = 1').fetchall()
for i in types_id:
    i = int(i[0])
    types.append(i)

fiction = []
clas = ['Offense', 'Defense', 'Support']
with sqlite3.connect('/Users/mac/Documents/Coding/Projects/DSA/Resources/dsa.db') as conn:
    curr = conn.cursor()
    fictions_id = curr.execute('SELECT category_id  FROM toons_categories where category_type = 4').fetchall()
for i in fictions_id:
    i = int(i[0])
    fiction.append(i)

story_id = []
with sqlite3.connect('/Users/mac/Documents/Coding/Projects/DSA/Resources/dsa.db') as conn:
    curr = conn.cursor()
    stories = curr.execute('SELECT category_id FROM toons_categories where category_type = 0').fetchall()
for i in stories:
    i = int(i[0])
    story_id.append(i)

clas_id = []
clas = ['Offense', 'Defense', 'Support']
with sqlite3.connect('/Users/mac/Documents/Coding/Projects/DSA/Resources/dsa.db') as conn:
    curr = conn.cursor()
    classes = curr.execute('SELECT category_id FROM toons_categories where category_type = 2').fetchall()
for i in classes:
    i = i[0]
    clas_id.append(int(i))
    

We want to keep our code as clean as possible, and We nned to question each categories list and to switch the data into our DataFrame, the following function fixes everything and returning a good to go Table

In [ ]:
def cat_fix(stg_categories,stg_toon_df):
    b4_shape = stg_toon_df.shape
    for cat in stg_categories:
        if cat in fiction:
            cat_name = unit_categories[f"{cat}"]['NameKey']
            cat_name =  cat_name.replace('UNIT_CATEGORY_','').replace('_',' ').title()
            stg_toon_df['Fiction'] = cat_name
        elif cat in story_id:
            cat_name = unit_categories[f"{cat}"]['NameKey']
            cat_name =  cat_name.replace('UNIT_CATEGORY_','').replace('_',' ').title()
            stg_toon_df['Story'] = cat_name
        elif cat in clas_id:
            cat_name = unit_categories[f"{cat}"]['NameKey']
            cat_name =  cat_name.replace('UNIT_CATEGORY_','').replace('_',' ').title()
            stg_toon_df['Class'] = cat_name
        elif cat in types:
            cat_name = unit_categories[f"{cat}"]['NameKey']
            cat_name =  cat_name.replace('UNIT_CATEGORY_','').replace('_',' ').title()
            stg_toon_df['Type'] = cat_name
    return stg_toon_df_c
        

And here the real thing:
We loop over all the toons and fixing them into normalized table, keeping the info we wants and everything as dast as possible.

In [ ]:
with sqlite3.connect('/Users/mac/Documents/Coding/Projects/DSA/Resources/dsa.db') as conn:
    for toon in chars_json:
        rel_cols = ['ID', 'NameKey','Obtainable', 'UnlockRarity','StatProgressionTable']
        namekey = chars_json[toon]['NameKey'].replace('_UNIT_NAME','').replace('_',' ').title()
        toon_df = pd.DataFrame.from_dict(chars_json[toon], orient='index').transpose()
        categories_series = toon_df['Categories']
        stg_toon_df_c = toon_df[rel_cols]
        stg_toon_df_c['NameKey'] = namekey
        stg_categories_c = chars_json[toon]['Categories']
        stg_toon_df_c = cat_fix(stg_toon_df=stg_toon_df_c,stg_categories=stg_categories_c)
        stg_toon_df_c.to_sql('toons',conn, if_exists='append',index=False)


    

In [ ]:
conn.commit()

And that will be all !
We did all the way from nested JSON object towards a normalized table that fixes all the toons. 